In [1]:
import sys
import requests
import tarfile
import json
import numpy as np
import pdf2image
from os import path
from wand.image import Image
from PIL import Image as PIL_Image
import xml.etree.ElementTree as ET
from PIL import ImageFont, ImageDraw
from glob import glob
from matplotlib import pyplot as plt
from pdf2image import convert_from_path
from PyPDF2 import PdfFileReader
from IPython.core.display import display, HTML
import pdb
import copy
import cv2
import pickle
import os
import traceback
from pprint import pprint
%matplotlib inline

In [2]:

dataset_type = 'train'
data_type_prefix = dataset_type + "_"
try:
    os.makedirs(data_type_prefix + 'images/')
except:
    print("Error in creating " + data_type_prefix + 'images/')
try:
    os.makedirs(data_type_prefix + 'bbox_images/')
except:
    print("Error in creating " + data_type_prefix + 'bbox_images/')
try:
    os.makedirs(data_type_prefix + 'intermediate_bbox_images/')
except:
    print("Error in creating " + data_type_prefix + 'intermediate_bbox_images/')
try:
    os.makedirs(data_type_prefix + 'original_bbox_images/')
except:
    print("Error in creating " + data_type_prefix + 'original_bbox_images/')
try:
    os.makedirs(data_type_prefix + 'xmls/')
except:
    print("Error in creating " + data_type_prefix + 'xmls/')


In [3]:
def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    return (matrix[size_x - 1, size_y - 1])

In [4]:
def create_root(table_img_path, file_prefix, width, height, depth):
    root = ET.Element("annotations")
    ET.SubElement(root, "folder").text = "images"
    ET.SubElement(root, "filename").text = "{}".format(file_prefix + '.png')
    ET.SubElement(
        root, "path").text = table_img_path + "{}".format(file_prefix)
    source = ET.SubElement(root, "source")
    ET.SubElement(source, "database").text = "Unknown"
    size = ET.SubElement(root, "size")
    ET.SubElement(size, "width").text = str(width)
    ET.SubElement(size, "height").text = str(height)
    ET.SubElement(size, "depth").text = str(depth)
    ET.SubElement(root, "segmentated").text = "0"
    return root

def create_object_annotation(root, table_coordinates, table_cells):
    #for i in range (length_table_list):
    obj = ET.SubElement(root, "object")
    ET.SubElement(obj, "name").text = "table"
    ET.SubElement(obj, "pose").text = "Unspecified"
    ET.SubElement(obj, "truncated").text = str(0)
    ET.SubElement(obj, "difficult").text = str(0)
    bbox = ET.SubElement(obj, "bndbox")
    ET.SubElement(bbox, "xmin").text = str(table_coordinates[0])

    ET.SubElement(bbox, "ymin").text = str(table_coordinates[2])
    ET.SubElement(bbox, "xmax").text = str(table_coordinates[1])
    ET.SubElement(bbox, "ymax").text = str(table_coordinates[3])
    cells = ET.SubElement(obj, "cells")
    for j in range(len(table_cells)):
        cell_detail = table_cells[j]
        if cell_detail["coordinates"] is None or len(cell_detail["coordinates"]) != 4:
            continue
        cell = ET.SubElement(cells, "tablecell")
        ET.SubElement(cell, "dont_care").text = "False"
        ET.SubElement(cell,
                        "end_col").text = str(cell_detail["end_col"])
        ET.SubElement(cell,
                        "end_row").text = str(cell_detail["end_row"])
        ET.SubElement(cell,
                        "start_col").text = str(cell_detail["start_col"])
        ET.SubElement(cell,
                        "start_row").text = str(cell_detail["start_row"])
        ET.SubElement(cell,
                        "x0").text = str(cell_detail["coordinates"][0])
        ET.SubElement(cell,
                        "x1").text = str(cell_detail["coordinates"][2])
        ET.SubElement(cell,
                        "y0").text = str(cell_detail["coordinates"][1])
        ET.SubElement(cell,
                        "y1").text = str(cell_detail["coordinates"][3])
    return root

In [5]:
def identify_column_indentations(table_cells_list, same_indent_threshold=5):
    column_indentations = {}
    column_starts = {}
    column_ends = {}
    for cell in table_cells_list:
        if cell['coordinates'] is None:
            continue
        start_col = cell['start_col']
        end_col = cell['end_col']
        x1, y1, x2, y2 = cell['coordinates']
        if start_col not in column_indentations:
            column_indentations[start_col] = 'center'
        if end_col not in column_indentations:
            column_indentations[end_col] = 'center'
        if start_col not in column_starts:
            if x1 is not None:
                column_starts[start_col] = {'min' : x1, 'max' : x1}
        if end_col not in column_ends:
            if x2 is not None:
                column_ends[end_col] = {'min' : x2, 'max' : x2}
        if x1 is not None:
            column_starts[start_col]['min'] = min(column_starts[start_col]['min'], x1)
            column_starts[start_col]['max'] = max(column_starts[start_col]['min'], x1)
        if x2 is not None:
            column_ends[end_col]['min'] = min(column_ends[end_col]['min'], x2)
            column_ends[end_col]['min'] = min(column_ends[end_col]['min'], x2)
        
    for column_id in column_indentations:
        try:
            if column_starts[column_id]['max'] - column_starts[column_id]['min'] < same_indent_threshold:
                if column_ends[column_id]['max'] - column_ends[column_id]['min'] >= same_indent_threshold:
                    column_indentations[column_id] = 'left'
            if column_ends[column_id]['max'] - column_ends[column_id]['min'] < same_indent_threshold:
                if column_starts[column_id]['max'] - column_starts[column_id]['min'] >= same_indent_threshold:
                    column_indentations[column_id] = 'right'
        except:
            column_indentations[column_id] = 'center'
    return column_indentations

In [6]:
def preprocess_cells(table_cells_list, table_coordinates, padding=10, column_indentations=None):
    if column_indentations is not None:
        left_indentation_ratio = 0.85
        right_indentation_ratio = 0.15
    default_cell_padding = 8
    missing_cells_padding_diff_ratio = 0.1
    output_table_cells_list = []
    output_table_cells_list_intermediate = []
    output_table_cells_list_original = []
    cells_xmin = {}
    cells_xmax = {}
    cells_ymin = {}
    cells_ymax = {}
    row_ymin = {}
    row_ymax = {}
    col_xmin = {}
    col_xmax = {}
    row_ymin_processed = {}
    row_ymax_processed = {}
    col_xmin_processed = {}
    col_xmax_processed = {}
    for cell_detail in table_cells_list:
        start_row = cell_detail['start_row']
        start_col = cell_detail['start_col']
        end_row = cell_detail['end_row']
        end_col = cell_detail['end_col']
        coords = cell_detail['coordinates']
        if coords is None:
            continue
        if coords[1] is not None:
            if start_row not in row_ymin:
                row_ymin[start_row] = coords[1] - table_coordinates[1]
            else:
                row_ymin[start_row] = min(row_ymin[start_row], coords[1] - table_coordinates[1])
        if coords[3] is not None:
            if end_row not in row_ymax:
                row_ymax[end_row] = coords[3] - table_coordinates[1]
            else:
                row_ymax[end_row] = max(row_ymax[end_row], coords[3] - table_coordinates[1])
        if coords[0] is not None:
            if start_col not in col_xmin:
                col_xmin[start_col] = coords[0] - table_coordinates[0]
            else:
                col_xmin[start_col] = min(col_xmin[start_col], coords[0] - table_coordinates[0])
        if coords[2] is not None:
            if end_col not in col_xmax:
                col_xmax[end_col] = coords[2] - table_coordinates[0]
            else:
                col_xmax[end_col] = max(col_xmax[end_col], coords[2] - table_coordinates[0])
                
    num_cols = max(len(col_xmin), len(col_xmax))
    num_rows = max(len(row_ymin), len(row_ymax))
                
    #Try to populate None coordinates
            
            
    #print(row_ymin, row_ymax, col_xmin, col_xmax)
    table_cells_list_original = copy.deepcopy(table_cells_list)
    for i in range(len(table_cells_list_original)):
        cell_detail = table_cells_list_original[i]
        coor = cell_detail['coordinates']
        if coor is None:
            continue
        coords = []
        if coor[0] is None or coor[1] is None or coor[2] is None or coor[3] is None:
            continue
        
        coords = [
            coor[0] - table_coordinates[0],
            coor[1] - table_coordinates[1],
            coor[2] - table_coordinates[0],
            coor[3] - table_coordinates[1]
        ]
        cell_detail['coordinates'] = coords
        output_table_cells_list_original.append(cell_detail)
    
#     min_row_id = min(min(list(row_ymin.keys())), min(list(row_ymax.keys())))
#     max_row_id = max(max(list(row_ymin.keys())), max(list(row_ymax.keys())))
#     for row_id in range(min_row_id, max_row_id + 1):
#         if row_id not in row_ymax:
#             if row_id + 1 in row_ymin:
#                 row_ymax[row_id] = row_ymin[row_id + 1] - default_cell_padding
#         if row_id not in row_ymin:
#             if row_id - 1 in row_ymax:
#                 row_ymin[row_id] = row_ymax[row_id - 1] + default_cell_padding
#     min_col_id = min(min(list(col_xmin.keys())), min(list(col_xmax.keys())))
#     max_col_id = max(max(list(col_xmin.keys())), max(list(col_xmax.keys())))
#     for col_id in range(min_col_id, max_col_id + 1):
#         if col_id not in col_xmax:
#             if col_id + 1 in col_xmin:
#                 col_xmax[col_id] = col_xmin[col_id + 1] - default_cell_padding
#         if col_id not in col_xmin:
#             if col_id - 1 in col_xmax:
#                 col_xmin[col_id] = col_xmax[col_id - 1] + default_cell_padding
            
            
    table_cells_list_intermediate = copy.deepcopy(table_cells_list)
    for i in range(len(table_cells_list_intermediate)):
        cell_detail = table_cells_list_intermediate[i]
        start_row = cell_detail['start_row']
        start_col = cell_detail['start_col']
        end_row = cell_detail['end_row']
        end_col = cell_detail['end_col']
        coords = []
        if start_col in col_xmin and start_row in row_ymin and end_col in col_xmax and end_row in row_ymax:
            coords = [
                col_xmin[start_col],
                row_ymin[start_row],
                col_xmax[end_col],
                row_ymax[end_row]
            ]
        cell_detail['coordinates'] = coords
        output_table_cells_list_intermediate.append(cell_detail)
        
    
#     print(col_xmin, col_xmax)
#     print(row_ymin, row_ymax)
    row_ymin_processed[min(list(row_ymin.keys()))] = row_ymin[min(list(row_ymin.keys()))] - padding
    row_ymax_processed[max(list(row_ymax.keys()))] = row_ymax[max(list(row_ymax.keys()))] + padding
    col_xmin_processed[min(list(col_xmin.keys()))] = col_xmin[min(list(col_xmin.keys()))] - padding
    col_xmax_processed[max(list(col_xmax.keys()))] = col_xmax[max(list(col_xmax.keys()))] + padding
    prev_index = min(list(col_xmax.keys()))
    cur_index = None
    for i in range(1, max(list(col_xmin.keys())) + 1):
        if prev_index not in col_xmax:
            prev_index += 1
            continue
        if i not in col_xmin or i <= prev_index:
            continue
        cur_index = i
        if column_indentations is not None and cur_index in column_indentations and column_indentations[cur_index] == 'left':
            diff = int(float((col_xmin[cur_index] - col_xmax[prev_index])) * left_indentation_ratio)
        elif column_indentations is not None and prev_index in column_indentations and column_indentations[prev_index] == 'right':
            diff = int(float((col_xmin[cur_index] - col_xmax[prev_index])) * right_indentation_ratio)
        else:
            diff = int(float((col_xmin[cur_index] - col_xmax[prev_index])) / 2.0)
            #diff = int(float((col_xmin[i] - col_xmax[i - 1])) / 2.0)
        col_xmax_processed[prev_index] = col_xmax[prev_index] + diff
        col_xmin_processed[cur_index] = col_xmax_processed[prev_index]
        prev_index = cur_index
            #col_xmin_processed[i] = col_xmin[i] - diff
    
    prev_index = min(list(row_ymax.keys()))
    cur_index = None
    for i in range(1, max(list(row_ymin.keys())) + 1):
        if i not in row_ymin or i <= prev_index:
            continue
        cur_index = i
        diff = int(float((row_ymin[cur_index] - row_ymax[prev_index])) / 2.0)
        row_ymax_processed[prev_index] = row_ymax[prev_index] + diff
        row_ymin_processed[cur_index] = row_ymax_processed[prev_index]
        prev_index = cur_index
        #row_ymin_processed[i] = row_ymin[i] - diff
    #print(col_xmin_processed, row_ymin_processed, col_xmax_processed, row_ymax_processed)
    
#     print(col_xmin_processed, col_xmax_processed)
#     print(row_ymin_processed, row_ymax_processed)
    is_row_assignment_valid = True
    is_col_assignment_valid = True
    for i in range(1, len(row_ymax_processed)):
        if i in row_ymax_processed and i-1 in row_ymax_processed:
            if row_ymax_processed[i] <= row_ymax_processed[i-1]:
                is_row_assignment_valid = False
        if i in row_ymin_processed and i-1 in row_ymin_processed:
            if row_ymin_processed[i] <= row_ymin_processed[i-1]:
                is_row_assignment_valid = False
    for i in range(1, len(col_xmax_processed)):
        if i in col_xmax_processed and i-1 in col_xmax_processed:
            if col_xmax_processed[i] <= col_xmax_processed[i-1]:
                is_col_assignment_valid = False
        if i in col_xmin_processed and i-1 in col_xmin_processed:
            if col_xmin_processed[i] <= col_xmin_processed[i-1]:
                is_col_assignment_valid = False
    
    ordering_failure = False
    if is_row_assignment_valid == False or is_col_assignment_valid == False:
        ordering_failure = True
    
    for i in range(len(table_cells_list)):
        cell_detail = table_cells_list[i]
        start_row = cell_detail['start_row']
        start_col = cell_detail['start_col']
        end_row = cell_detail['end_row']
        end_col = cell_detail['end_col']
        coords = []
        #if start_col in col_xmin_processed and start_row in row_ymin_processed and end_col in col_xmax_processed and end_row in row_ymax_processed:
        if start_col in col_xmin_processed:
            coords.append(col_xmin_processed[start_col])
        elif start_col in col_xmin:
            coords.append(col_xmin[start_col] - default_cell_padding)
        if start_row in row_ymin_processed:
            coords.append(row_ymin_processed[start_row])
        elif start_row in row_ymin:
            coords.append(row_ymin[start_row] - default_cell_padding)
        if end_col in col_xmax_processed:
            coords.append(col_xmax_processed[end_col])
        elif end_col in col_xmax:
            coords.append(col_xmax[end_col] + default_cell_padding)
        if end_row in row_ymax_processed:
            coords.append(row_ymax_processed[end_row])
        elif end_row in row_ymax:
            coords.append(row_ymax[end_row] + default_cell_padding)
        if len(coords) != 4:
            continue
        cell_detail['coordinates'] = coords
        output_table_cells_list.append(cell_detail)
    return (output_table_cells_list, output_table_cells_list_intermediate, output_table_cells_list_original), (num_rows, num_cols), ordering_failure

In [7]:
def map_pdf_to_image_coordinates(pdf_image, png_image, src_coordinates, page_no=0):
    sx0, sy0, sx1, sy1 = src_coordinates
    pdf = pdf_image.sequence[page_no]
    im = png_image.sequence[page_no]
    ssx, ssy = (pdf.size[0], pdf.size[1])
    #print(ssx, ssy)
    dsx, dsy = im.size
    if ssx > ssy:
        tmp = ssx
        ssx = ssy
        ssy = tmp
    if dsx > dsy:
        tmp = dsx
        dsx = dsy
        dsy = tmp
    #sy01 = sy0
    #sy11 = sy1
    sy01 = ssy - sy1
    sy11 = ssy - sy0
    if sy01 < 0 or sy11 < 0:
        #print(ssx, ssy, dsx, dsy)
        #print(sx0, sy0, sx1, sy1)
        #print(sy01, sy11)
        pass
    x0 = int((sx0 * dsx) / ssx)
    x1 = int((sx1 * dsx) / ssx)
    y0 = int((sy01 * dsy) / ssy)
    y1 = int((sy11 * dsy) / ssy)
    #print(x0, y0, x1, y1)
    return [x0, y0, x1, y1]

In [8]:
def get_cv_png_and_pdf(pdf_file_path):
    cells = []
    image_original_pdf = Image(filename=pdf_file_path)
    image_pdf = Image(filename=pdf_file_path, resolution=300)
    image_png = image_pdf.convert('png')
    Image(image_png.sequence[0]).save(filename=data_type_prefix + 'tmp.png')
    cv_image = cv2.imread(data_type_prefix + 'tmp.png')
    return cv_image, image_png, image_original_pdf

In [9]:
def get_structure_and_chunks(structure_file_path, chunk_file_path):
    chunks = None
    structure = None
    with open(chunk_file_path, 'r') as cf:
        chunks = json.loads(cf.read())
        chunks = chunks['chunks']
    with open(structure_file_path, 'r') as sf:
        structure = json.loads(sf.read())
        structure = structure['cells']
    structure = sorted(structure, key=lambda k: k['id'])
    match_structure = {}
    match_chunks = {}
    index = 0
    empty_cell_index = 0
    for cell in structure:
        match_cell = cell.copy()
        empty_cell = False
        if len(match_cell['content']) == 0:
            empty_cell_index += 1
            empty_cell = True
        match_cell['match_content'] = ''.join(match_cell['content'])
        match_cell['index'] = index
        match_cell['matched'] = False
        if empty_cell:
            match_cell['match_index'] = -1
        else:
            match_cell['match_index'] = index - empty_cell_index
        match_structure[index] = match_cell 
        index += 1
    index = 0
    for chunk in chunks:
        match_chunk = chunk.copy()
        coordinates_x0_x1_y0_y1 = chunk['pos']
        coordinates_x0_y0_x1_y1 = np.asarray([coordinates_x0_x1_y0_y1[0], coordinates_x0_x1_y0_y1[2], coordinates_x0_x1_y0_y1[1], coordinates_x0_x1_y0_y1[3]])
        match_chunk['pos'] = coordinates_x0_y0_x1_y1
        match_chunk['match_content'] = match_chunk['text'].replace(' ', '')
        match_chunk['index'] = index
        match_chunk['matched'] = False
        match_chunks[index] = match_chunk
        index += 1
    return match_structure, match_chunks
    

In [10]:
def merge_structure_and_chunks(structure, chunks, matches):
    cells = []
    index = 0
    for cell_index in sorted(structure.keys()):
        if structure[cell_index]['matched'] == True:
            cell_details = {
                "cell_id" : index,
                "coordinates" : chunks[matches[cell_index]]['pos'],
                "content" : " ".join(structure[cell_index]['content']),
                "start_row" : structure[cell_index]['start_row'],
                "end_row" : structure[cell_index]['end_row'],
                "start_col" : structure[cell_index]['start_col'],
                "end_col" : structure[cell_index]['end_col']
            }
        else:
            cell_details = {
                "cell_id" : index,
                "coordinates" : None,
                "content" : " ".join(structure[cell_index]['content']),
                "start_row" : structure[cell_index]['start_row'],
                "end_row" : structure[cell_index]['end_row'],
                "start_col" : structure[cell_index]['start_col'],
                "end_col" : structure[cell_index]['end_col']
            }
        index += 1
        cells.append(cell_details)
    return cells

def match_structure_and_chunks(structure, chunks, matches={}, search_proximity = 50, min_edit_distance_threshold=0, run_id=0):
    matched_flag = False
    last_chunk_index_matched = -1
    for cell_index in structure:
        cell = structure[cell_index]
        if cell['match_index'] == -1:
            continue
        if cell['matched'] == True:
            continue
        cell_id = cell['match_index']
        start_chunk_index = max(0, cell_id - search_proximity)
        end_chunk_index = min(cell_index + search_proximity, len(chunks))
        edit_distance_indices = {}
        all_matched_indices = []
        for chunk_index in range(start_chunk_index, end_chunk_index):
            if chunks[chunk_index]['matched'] == True:
                continue
            if chunk_index <= last_chunk_index_matched:
                continue
            edit_distance = levenshtein(structure[cell_index]['match_content'], chunks[chunk_index]['match_content'])
            if edit_distance > min_edit_distance_threshold:
                continue
            if edit_distance <= min_edit_distance_threshold and run_id > 0:
                all_matched_indices.append(chunk_index - cell_index)
            if edit_distance not in edit_distance_indices:
                edit_distance_indices[edit_distance] = []
            edit_distance_indices[edit_distance].append(chunk_index)
        if len(edit_distance_indices) > 0 and run_id == 0:
            min_edit_distance = min(edit_distance_indices.keys())
            #print(cell_id, min_edit_distance, edit_distance_indices[min_edit_distance], min(edit_distance_indices[min_edit_distance], key=abs))
            best_matched_chunk_index = int(min(edit_distance_indices[min_edit_distance]))
            last_chunk_index_matched = best_matched_chunk_index
            chunks[best_matched_chunk_index]['matched'] = True
            structure[cell_index]['matched'] = True
            matches[cell_index] = best_matched_chunk_index
            matched_flag = True
        elif len(all_matched_indices) > 0 and run_id > 0:
            best_matched_chunk_index = int(min(all_matched_indices, key=abs)) + cell_index
            last_chunk_index_matched = best_matched_chunk_index
            chunks[best_matched_chunk_index]['matched'] = True
            structure[cell_index]['matched'] = True
            matches[cell_index] = best_matched_chunk_index
            matched_flag = True
    return matches, matched_flag, structure, chunks


def recursively_match_structure_and_chunks(structure, chunks, search_proximity = 50):
    min_edit_distance_threshold = 0
    search_proximity = 50
    matched_flag = True
    matches = {}
    run_id = 0
    while matched_flag:
        matches, matched_flag, structure, chunks = match_structure_and_chunks(structure, chunks, matches=matches, search_proximity = search_proximity, min_edit_distance_threshold = min_edit_distance_threshold, run_id=run_id)
        min_edit_distance_threshold += 50
        search_proximity = 20
        run_id += 1
#         pprint("*******************")
#         pprint(matches)
#         pprint(structure)
#         pprint(chunks)
    return matches

def get_table_cells(structure_file_path, chunk_file_path):
    structure, chunks = get_structure_and_chunks(structure_file_path, chunk_file_path)
    matches = recursively_match_structure_and_chunks(structure, chunks)
    cells = merge_structure_and_chunks(structure, chunks, matches)
    return cells

In [11]:
def establish_validity_of_matches(cells):
    #All unique rows and columns should have at-least one match of chunk
    #Also, for each row, x-coordinate-ordering of columns should match structure
    #And for each column, y-coordinate-ordering of columns should match structure
    rows_assigned = {}
    cols_assigned = {}
    for cell in cells:
        if cell['start_row'] not in rows_assigned:
            if cell['coordinates'] is not None and cell['start_row'] == cell['end_row']:
                rows_assigned[cell['start_row']] = True
            else:
                rows_assigned[cell['start_row']] = False
        else:
            if cell['coordinates'] is not None and cell['start_row'] == cell['end_row']:
                rows_assigned[cell['start_row']] = True
        if cell['start_col'] not in cols_assigned:
            if cell['coordinates'] is not None and cell['start_col'] == cell['end_col']:
                cols_assigned[cell['start_col']] = True
            else:
                cols_assigned[cell['start_col']] = False
        else:
            if cell['coordinates'] is not None and cell['start_col'] == cell['end_col']:
                cols_assigned[cell['start_col']] = True
    all_rows_assigned = True
#     print(rows_assigned, cols_assigned)
    for row_id in rows_assigned:
        all_rows_assigned = all_rows_assigned and rows_assigned[row_id]
    all_cols_assigned = True
    for col_id in cols_assigned:
        all_cols_assigned = all_cols_assigned and cols_assigned[col_id]
        
    #print(rows_assigned, cols_assigned)
    
#     col_start_x_coords = {}
#     col_end_x_coords = {}
#     for row_id in rows_assigned:
#         if row_id not in col_start_x_coords:
#             col_start_x_coords[row_id] = {}
#         if row_id not in col_end_x_coords:
#             col_end_x_coords[row_id] = {}
#     for cell in cells:
#         if cell['coordinates'] is None:
#             continue
#         for i in range(cell['start_row'], cell['end_row'] + 1):
#             col_start_x_coords[i][cell['start_col']] = cell['coordinates'][0]
#             col_end_x_coords[i][cell['end_col']] = cell['coordinates'][2]
#     row_start_y_coords = {}
#     row_end_y_coords = {}
#     for col_id in cols_assigned:
#         if col_id not in row_start_y_coords:
#             col_start_x_coords[col_id] = {}
#         if col_id not in row_end_y_coords:
#             col_end_x_coords[col_id] = {}
#     for cell in cells:
#         if cell['coordinates'] is None:
#             continue
#         for i in range(cell['start_col'], cell['end_col'] + 1):
#             row_start_y_coords[i][cell['start_row']] = cell['coordinates'][1]
#             row_end_y_coords[i][cell['end_row']] = cell['coordinates'][3]
    return all_rows_assigned and all_cols_assigned

In [12]:
def remove_column_bleeding_cordinates(cells):
    def identify_column_coordinates(cells):
        max_col_starts = {}
        min_col_ends = {}
        for cell in cells:
            if cell['coordinates'] is None:
                continue
            start_col = cell['start_col']
            end_col = cell['end_col']
            if start_col not in max_col_starts:
                max_col_starts[start_col] = cell['coordinates'][0]
            else:
                max_col_starts[start_col] = max(max_col_starts[start_col], cell['coordinates'][0])
            if end_col not in min_col_ends:
                min_col_ends[end_col] = cell['coordinates'][2]
            else:
                min_col_ends[end_col] = min(min_col_ends[end_col], cell['coordinates'][2])
        return max_col_starts, min_col_ends
    
    max_col_starts, min_col_ends = identify_column_coordinates(cells)
    bleed_flag = False
#     print(max_col_starts, min_col_ends)
    for i in range(len(cells)):
        for j in range(len(cells)):
            if i == j:
                continue
            if (cells[i]['start_row'] > cells[j]['end_row']) or (cells[i]['end_row'] < cells[j]['start_row']):
                continue
            if cells[i]['coordinates'] is None:
                continue
            if cells[i]['end_col'] < cells[j]['start_col'] and cells[j]['start_col'] in max_col_starts:
                if cells[i]['coordinates'][2] is not None and cells[i]['coordinates'][2] >= max_col_starts[cells[j]['start_col']]:
                    bleed_flag = True
                    cells[i]['coordinates'][2] = None
            if cells[i]['start_col'] > cells[j]['end_col'] and cells[j]['end_col'] in min_col_ends:
                if cells[i]['coordinates'][0] is not None and cells[i]['coordinates'][0] <= min_col_ends[cells[j]['end_col']]:
                    bleed_flag = True
                    cells[i]['coordinates'][0] = None
    return cells, bleed_flag

def remove_row_bleeding_cordinates(cells):
    def identify_row_coordinates(cells):
        max_row_starts = {}
        min_row_ends = {}
        for cell in cells:
            if cell['coordinates'] is None:
                continue
            start_row = cell['start_row']
            end_row = cell['end_row']
            if start_row not in max_row_starts:
                max_row_starts[start_row] = cell['coordinates'][1]
            else:
                max_row_starts[start_row] = max(max_row_starts[start_row], cell['coordinates'][1])
            if end_row not in min_row_ends:
                min_row_ends[end_row] = cell['coordinates'][3]
            else:
                min_row_ends[end_row] = min(min_row_ends[end_row], cell['coordinates'][3])
        return max_row_starts, min_row_ends
    
    max_row_starts, min_row_ends = identify_row_coordinates(cells)
    bleed_flag = False
    #print(max_row_starts, min_row_ends)
    for i in range(len(cells)):
        for j in range(len(cells)):
            if i == j:
                continue
            if (cells[i]['start_col'] > cells[j]['end_col']) or (cells[i]['end_col'] < cells[j]['start_col']):
                continue
            if cells[i]['coordinates'] is None:
                continue
            if cells[i]['end_row'] < cells[j]['start_row'] and cells[j]['start_row'] in max_row_starts:
                if cells[i]['coordinates'][3] is not None and cells[i]['coordinates'][3] >= max_row_starts[cells[j]['start_row']]:
                    bleed_flag = True
                    cells[i]['coordinates'][3] = None
            if cells[i]['start_row'] > cells[j]['end_row'] and cells[j]['end_row'] in min_row_ends:
                if cells[i]['coordinates'][1] is not None and cells[i]['coordinates'][1] <= min_row_ends[cells[j]['end_row']]:
                    bleed_flag = True
                    cells[i]['coordinates'][1] = None
    return cells, bleed_flag

In [13]:
def fill_missing_regions(cells):
    min_row_id = min(cell['start_row'] for cell in cells)
    min_col_id = min(cell['start_col'] for cell in cells)
    max_row_id = max(cell['end_row'] for cell in cells)
    max_col_id = max(cell['end_col'] for cell in cells)
    row_regions_by_col = {}
    col_regions_by_row = {}
    cell_indexes_by_structure = {}
    for cell in cells:
        for col_id in range(cell['start_col'], cell['end_col'] + 1):
            for row_id in range(cell['start_row'], cell['end_row'] + 1):
                cell_indexes_by_structure[(row_id, col_id)] = cell['cell_id']
                if col_id not in row_regions_by_col:
                    row_regions_by_col[col_id] = []
                if row_id not in col_regions_by_row:
                    col_regions_by_row[row_id] = []
                row_regions_by_col[col_id].append(row_id)
                col_regions_by_row[row_id].append(col_id)
    
#     print(row_regions_by_col, col_regions_by_row)
    for col_id in row_regions_by_col:
        row_regions = sorted(row_regions_by_col[col_id])
        if row_regions[0] > min_row_id:
            cells[cell_indexes_by_structure(row_regions[0], col_id)]['start_row'] = min_row_id
        for i in range(len(row_regions) - 1):
            row_id = row_regions[i]
            next_row_id = row_regions[i+1]
            if row_id + 1 != next_row_id:
                cells[cell_indexes_by_structure(row_id, col_id)]['end_row'] += next_row_id - row_id - 1
        row_id = row_regions[len(row_regions) - 1]
        if row_id != max_row_id:
            cells[cell_indexes_by_structure(row_id, col_id)]['end_row'] = max_row_id
            
    for row_id in col_regions_by_row:
        col_regions = sorted(col_regions_by_row[row_id])
        if col_regions[0] > min_col_id:
            cells[cell_indexes_by_structure(row_id, col_regions[0])]['start_col'] = min_col_id
        for i in range(len(col_regions) - 1):
            col_id = col_regions[i]
            next_col_id = col_regions[i+1]
            if col_id + 1 != next_col_id:
                cells[cell_indexes_by_structure(row_id, col_id)]['end_col'] += next_col_id - col_id - 1
        col_id = col_regions[len(col_regions) - 1]
        if col_id != max_col_id:
            cells[cell_indexes_by_structure(row_id, col_id)]['end_col'] = max_col_id
    return cells
        

In [14]:
def process_sample(file_prefix, 
                   structure_dir_path=dataset_type + "/structure/", 
                   chunk_dir_path=dataset_type + "/chunk/", 
                   pdf_dir_path=dataset_type+'/pdf/', 
                   padding=40):
    failure_code = -1
    structure_file_path = structure_dir_path + file_prefix + ".json"
    chunk_file_path = chunk_dir_path + file_prefix + ".chunk"
    pdf_file_path = pdf_dir_path + file_prefix + ".pdf"
    cv_image, image_png, image_original_pdf = get_cv_png_and_pdf(pdf_file_path) 
    pdf_cells = get_table_cells(structure_file_path, chunk_file_path)
    is_valid = establish_validity_of_matches(pdf_cells)
    if is_valid == False:
        print("Not all unique rows and columns assigned at least one bbox")
        failure_code = 1
    #pprint(pdf_cells)
    table_coordinates = [10000, 10000, -10, -10]
    cells = []
    num_empty_cells = 0
    for cell_index in range(len(pdf_cells)):
        cell_details = copy.deepcopy(pdf_cells[cell_index])
        if pdf_cells[cell_index]['coordinates'] is not None:
            mapped_coords = map_pdf_to_image_coordinates(image_original_pdf, image_png, pdf_cells[cell_index]['coordinates'])
        else:
            mapped_coords = None
        cell_details['coordinates'] = mapped_coords
        if cell_details['content'] == "":
            num_empty_cells += 1
        if mapped_coords is not None:
            if mapped_coords[0] < table_coordinates[0]:
                table_coordinates[0] = mapped_coords[0]
            if mapped_coords[1] < table_coordinates[1]:
                table_coordinates[1] = mapped_coords[1]
            if mapped_coords[2] > table_coordinates[2]:
                table_coordinates[2] = mapped_coords[2]
            if mapped_coords[3] > table_coordinates[3]:
                table_coordinates[3] = mapped_coords[3]
        cells.append(cell_details)
    
    table_coordinates[0] = max(0, table_coordinates[0] - padding)
    table_coordinates[1] = max(0, table_coordinates[1] - padding)
    table_coordinates[2] = min(cv_image.shape[1], table_coordinates[2] + padding)
    table_coordinates[3] = min(cv_image.shape[0], table_coordinates[3] + padding)
    cells, column_bleed_flag = remove_column_bleeding_cordinates(cells)
    cells, row_bleed_flag = remove_row_bleeding_cordinates(cells)
    #cells = fill_missing_regions(cells)
    column_indentations = identify_column_indentations(cells, same_indent_threshold=5)
    #column_indentations = None
    preprocess_cells_output, table_dims, ordering_failure = preprocess_cells(copy.deepcopy(cells), table_coordinates, int(padding/2), column_indentations=column_indentations)
    num_rows, num_cols = table_dims
    if ordering_failure is True:
        print("Ordering of coordinates is inconsistent with structure. Issue most likely in matching")
        failure_code = 2
    if num_rows == 1:
        print("Only a single row is present in the table.")
        failure_code = 3
    if num_cols == 1:
        print("Only a single column is present in the table.")
        failure_code = 4
    if column_bleed_flag == True:
        print("Column bleeding was observed and corrected.")
        failure_code = 5
    if row_bleed_flag == True:
        print("Row bleeding was observed and corrected.")
        failure_code = 6
    table_cells_list, intermediate_table_cells_list, original_cells_list = preprocess_cells_output
    failure_fname_prefix = ''
    if failure_code != -1:
        failure_fname_prefix = 'failure_' + str(failure_code) + "_"
    sample_details = {
        'filepath': pdf_file_path,
        'output_image_path' : data_type_prefix + 'images/' + failure_fname_prefix + file_prefix + '.png',
        'output_bbox_image_path' : data_type_prefix + 'bbox_images/' + failure_fname_prefix + file_prefix + '.png',
        'output_intermediate_bbox_image_path' : data_type_prefix + 'intermediate_bbox_images/' + failure_fname_prefix + file_prefix + '.png',
        'output_original_bbox_image_path' : data_type_prefix + 'original_bbox_images/' + failure_fname_prefix + file_prefix + '.png',
        'output_xml_path' : data_type_prefix + 'xmls/' + failure_fname_prefix + file_prefix + '.xml',
        'cells': table_cells_list, 
        'original_cells' : original_cells_list,
        'intermediate_cells' : intermediate_table_cells_list,
        'table_coordinates' : table_coordinates,
        'num_empty_cells' : num_empty_cells
    }

    cv_table_image = cv_image[table_coordinates[1]:table_coordinates[3],table_coordinates[0]:table_coordinates[2], :]
    cv_table_bbox_image = cv_table_image.copy()
    cv_table_orig_bbox_image = cv_table_image.copy()
    cv_table_intermediate_bbox_image = cv_table_image.copy()
    for cell_detail in table_cells_list:
        coords = cell_detail['coordinates']
        if len(coords) == 4:
            cv2.rectangle(cv_table_bbox_image, (int(coords[0]), int(coords[1])), (int(coords[2]), int(coords[3])), (255, 0, 0), thickness=2)
    cv2.imwrite(sample_details['output_image_path'], cv_table_image)
    cv2.imwrite(sample_details['output_bbox_image_path'], cv_table_bbox_image)

    for cell_detail in original_cells_list:
        coords = cell_detail['coordinates']
        if coords is None:
            continue
        if len(coords) == 4:
             cv2.rectangle(cv_table_orig_bbox_image, (int(coords[0]), int(coords[1])), (int(coords[2]), int(coords[3])), (0, 255, 0), thickness=2)
    cv2.imwrite(sample_details['output_original_bbox_image_path'], cv_table_orig_bbox_image)

    for cell_detail in intermediate_table_cells_list:
        coords = cell_detail['coordinates']
        if len(coords) == 4:
            cv2.rectangle(cv_table_intermediate_bbox_image, (int(coords[0]), int(coords[1])), (int(coords[2]), int(coords[3])), (0, 0, 255), thickness=2)
    cv2.imwrite(sample_details['output_intermediate_bbox_image_path'], cv_table_intermediate_bbox_image)


    root = create_root(data_type_prefix + 'images/', file_prefix + '.png', cv_table_image.shape[1],
                                   cv_table_image.shape[0], cv_table_image.shape[2])
    root = create_object_annotation(root, [0, 0, cv_table_image.shape[1], cv_table_image.shape[0]], table_cells_list)
    tree = ET.ElementTree(root)
    tree.write(sample_details['output_xml_path'])
    return sample_details, failure_code
        
    

In [15]:
def run_single_sample(test_file_prefix):
    structure_file_path = dataset_type + '/structure/' + test_file_prefix + '.json'
    chunk_file_path = dataset_type + '/chunk/' + test_file_prefix + '.chunk'
    structure, chunks = get_structure_and_chunks(structure_file_path, chunk_file_path)
    pprint(structure)
    pprint(chunks)
    matches = recursively_match_structure_and_chunks(structure, chunks)
    pprint(matches)
    pprint(process_sample(test_file_prefix))

In [16]:
def run_dataset_execution():    
    base_dir = dataset_type + '/pdf/'
    index = 0
    break_after=None
    error_counts = 0
    runtime_exception_counts = 0
    failure_code_but_output_generated_counts = 0
    failures = {1 : 
                {
                    "type" : "Not all unique rows and columns assigned at least one bbox", 
                    "files" : [],
                    "counts" : 0
                },
                2 :
                {
                    "type" : "Ordering of coordinates is inconsistent with structure. Issue most likely in matching",
                    "files" : [],
                    "counts" : 0
                },
                3 :
                {
                    "type" : "Only a single row is present in the table.",
                    "files" : [],
                    "counts" : 0
                },
                4 :
                {
                    "type" : "Only a single column is present in the table.",
                    "files" : [],
                    "counts" : 0
                },
                5 :
                {
                    "type" : "Column bleeding was observed and corrected.",
                    "files" : [],
                    "counts" : 0
                },
                6 :
                {
                    "type" : "Row bleeding was observed and corrected.",
                    "files" : [],
                    "counts" : 0
                },
                7 :
                {
                    "type" : "Runtime Exception",
                    "files" : [],
                    "counts" : 0
                }
               }
    for pdf_file in os.listdir(base_dir):
        file_prefix = ".".join(pdf_file.split('.')[:-1])
        print("\nProcessing file " + str(index + 1) + " : " + file_prefix)
        print("\t Total Error Counts till now : " + str(error_counts))
        print("\t Total Counts where failure encountered but output generated till now : " + str(failure_code_but_output_generated_counts))
        print("\t\t", failures[1]['counts'], failures[2]['counts'])
        print("\t Total Runtime Exception Counts  till now : " + str(runtime_exception_counts))
        try:
            output, failure_code = process_sample(file_prefix)
            if failure_code == -1:
                print("Successfully processed: " + file_prefix)
            else:
                print("Failure code encountered but output generated: " + file_prefix)
                failures[failure_code]['files'].append(file_prefix)
                failures[failure_code]['counts'] += 1
                error_counts += 1
                failure_code_but_output_generated_counts += 1
        except:
            print("Runtime exception in processing: " + file_prefix)
            failures[3]['files'].append(file_prefix)
            failures[3]['counts'] += 1
            error_counts += 1
            runtime_exception_counts += 1
        index += 1
        if break_after is not None and index >= break_after:
            break
    import pickle
    pickle.dump(failures, open(dataset_type + "_failures.pickle", "wb"))

In [17]:
run_dataset_execution()


Processing file 1 : 1102.0674v1.1
	 Total Error Counts till now : 0
	 Total Counts where failure encountered but output generated till now : 0
		 0 0
	 Total Runtime Exception Counts  till now : 0
Column bleeding was observed and corrected.
Failure code encountered but output generated: 1102.0674v1.1

Processing file 2 : 1712.04604v3.1
	 Total Error Counts till now : 1
	 Total Counts where failure encountered but output generated till now : 1
		 0 0
	 Total Runtime Exception Counts  till now : 0
Successfully processed: 1712.04604v3.1

Processing file 3 : 1707.05260v4.1
	 Total Error Counts till now : 1
	 Total Counts where failure encountered but output generated till now : 1
		 0 0
	 Total Runtime Exception Counts  till now : 0
Successfully processed: 1707.05260v4.1

Processing file 4 : 1505.04891v2.3
	 Total Error Counts till now : 1
	 Total Counts where failure encountered but output generated till now : 1
		 0 0
	 Total Runtime Exception Counts  till now : 0
Successfully processed

Successfully processed: 1707.07494v1.1

Processing file 34 : 1608.04812v2.1
	 Total Error Counts till now : 4
	 Total Counts where failure encountered but output generated till now : 4
		 1 0
	 Total Runtime Exception Counts  till now : 0
Successfully processed: 1608.04812v2.1

Processing file 35 : 1705.03385v1.2
	 Total Error Counts till now : 4
	 Total Counts where failure encountered but output generated till now : 4
		 1 0
	 Total Runtime Exception Counts  till now : 0
Successfully processed: 1705.03385v1.2

Processing file 36 : 1612.08207v1.5
	 Total Error Counts till now : 4
	 Total Counts where failure encountered but output generated till now : 4
		 1 0
	 Total Runtime Exception Counts  till now : 0
Successfully processed: 1612.08207v1.5

Processing file 37 : 1701.06547v5.6
	 Total Error Counts till now : 4
	 Total Counts where failure encountered but output generated till now : 4
		 1 0
	 Total Runtime Exception Counts  till now : 0
Only a single column is present in the table

Not all unique rows and columns assigned at least one bbox
Failure code encountered but output generated: 1401.0778v1.1

Processing file 68 : 1603.09423v1.3
	 Total Error Counts till now : 7
	 Total Counts where failure encountered but output generated till now : 7
		 2 0
	 Total Runtime Exception Counts  till now : 0
Successfully processed: 1603.09423v1.3

Processing file 69 : 1611.05125v3.3
	 Total Error Counts till now : 7
	 Total Counts where failure encountered but output generated till now : 7
		 2 0
	 Total Runtime Exception Counts  till now : 0
Successfully processed: 1611.05125v3.3

Processing file 70 : 1605.05619v1.1
	 Total Error Counts till now : 7
	 Total Counts where failure encountered but output generated till now : 7
		 2 0
	 Total Runtime Exception Counts  till now : 0
Successfully processed: 1605.05619v1.1

Processing file 71 : 1804.06701v1.3
	 Total Error Counts till now : 7
	 Total Counts where failure encountered but output generated till now : 7
		 2 0
	 Total Ru

Successfully processed: 1704.06380v1.4

Processing file 103 : 1707.07657v2.16
	 Total Error Counts till now : 7
	 Total Counts where failure encountered but output generated till now : 7
		 2 0
	 Total Runtime Exception Counts  till now : 0
Successfully processed: 1707.07657v2.16

Processing file 104 : 1604.07236v3.2
	 Total Error Counts till now : 7
	 Total Counts where failure encountered but output generated till now : 7
		 2 0
	 Total Runtime Exception Counts  till now : 0
Successfully processed: 1604.07236v3.2

Processing file 105 : 1703.02819v1.2
	 Total Error Counts till now : 7
	 Total Counts where failure encountered but output generated till now : 7
		 2 0
	 Total Runtime Exception Counts  till now : 0
Successfully processed: 1703.02819v1.2

Processing file 106 : 1807.07255v2.5
	 Total Error Counts till now : 7
	 Total Counts where failure encountered but output generated till now : 7
		 2 0
	 Total Runtime Exception Counts  till now : 0
Successfully processed: 1807.07255v2.5

Successfully processed: 1709.09118v5.3

Processing file 137 : 1808.01104v1.3
	 Total Error Counts till now : 9
	 Total Counts where failure encountered but output generated till now : 9
		 3 0
	 Total Runtime Exception Counts  till now : 0
Successfully processed: 1808.01104v1.3

Processing file 138 : 1612.04418v1.3
	 Total Error Counts till now : 9
	 Total Counts where failure encountered but output generated till now : 9
		 3 0
	 Total Runtime Exception Counts  till now : 0
Successfully processed: 1612.04418v1.3

Processing file 139 : 1807.05519v1.22
	 Total Error Counts till now : 9
	 Total Counts where failure encountered but output generated till now : 9
		 3 0
	 Total Runtime Exception Counts  till now : 0
Successfully processed: 1807.05519v1.22

Processing file 140 : 1706.02929v1.1
	 Total Error Counts till now : 9
	 Total Counts where failure encountered but output generated till now : 9
		 3 0
	 Total Runtime Exception Counts  till now : 0
Successfully processed: 1706.02929v1.1

...

Successfully processed: 1801.03710v1.1

Processing file 11858 : 1705.04916v1.3
	 Total Error Counts till now : 715
	 Total Counts where failure encountered but output generated till now : 701
		 231 2
	 Total Runtime Exception Counts  till now : 14
Successfully processed: 1705.04916v1.3

Processing file 11859 : 1702.08563v2.2
	 Total Error Counts till now : 715
	 Total Counts where failure encountered but output generated till now : 701
		 231 2
	 Total Runtime Exception Counts  till now : 14
Successfully processed: 1702.08563v2.2

Processing file 11860 : 1503.01881v2.1
	 Total Error Counts till now : 715
	 Total Counts where failure encountered but output generated till now : 701
		 231 2
	 Total Runtime Exception Counts  till now : 14
Successfully processed: 1503.01881v2.1

Processing file 11861 : 1608.00944v1.3
	 Total Error Counts till now : 715
	 Total Counts where failure encountered but output generated till now : 701
		 231 2
	 Total Runtime Exception Counts  till now : 14
Succ

Successfully processed: 1705.06338v1.2

Processing file 11891 : 1612.00729v1.5
	 Total Error Counts till now : 718
	 Total Counts where failure encountered but output generated till now : 704
		 232 2
	 Total Runtime Exception Counts  till now : 14
Successfully processed: 1612.00729v1.5

Processing file 11892 : 1012.5030v1.6
	 Total Error Counts till now : 718
	 Total Counts where failure encountered but output generated till now : 704
		 232 2
	 Total Runtime Exception Counts  till now : 14
Successfully processed: 1012.5030v1.6

Processing file 11893 : 1711.05861v1.4
	 Total Error Counts till now : 718
	 Total Counts where failure encountered but output generated till now : 704
		 232 2
	 Total Runtime Exception Counts  till now : 14
Successfully processed: 1711.05861v1.4

Processing file 11894 : 1606.00629v2.1
	 Total Error Counts till now : 718
	 Total Counts where failure encountered but output generated till now : 704
		 232 2
	 Total Runtime Exception Counts  till now : 14
Column

Successfully processed: 1801.06391v1.1

Processing file 11924 : 1706.07238v1.8
	 Total Error Counts till now : 720
	 Total Counts where failure encountered but output generated till now : 706
		 232 2
	 Total Runtime Exception Counts  till now : 14
Successfully processed: 1706.07238v1.8

Processing file 11925 : 1602.06359v1.3
	 Total Error Counts till now : 720
	 Total Counts where failure encountered but output generated till now : 706
		 232 2
	 Total Runtime Exception Counts  till now : 14
Successfully processed: 1602.06359v1.3

Processing file 11926 : 1504.06700v1.1
	 Total Error Counts till now : 720
	 Total Counts where failure encountered but output generated till now : 706
		 232 2
	 Total Runtime Exception Counts  till now : 14
Successfully processed: 1504.06700v1.1

Processing file 11927 : 0906.2767v1.1
	 Total Error Counts till now : 720
	 Total Counts where failure encountered but output generated till now : 706
		 232 2
	 Total Runtime Exception Counts  till now : 14
Succe

Successfully processed: 1612.08354v1.1

Processing file 11957 : 1607.04378v1.4
	 Total Error Counts till now : 720
	 Total Counts where failure encountered but output generated till now : 706
		 232 2
	 Total Runtime Exception Counts  till now : 14
Successfully processed: 1607.04378v1.4

Processing file 11958 : 1711.08937v3.1
	 Total Error Counts till now : 720
	 Total Counts where failure encountered but output generated till now : 706
		 232 2
	 Total Runtime Exception Counts  till now : 14
Successfully processed: 1711.08937v3.1

Processing file 11959 : 1704.01074v4.3
	 Total Error Counts till now : 720
	 Total Counts where failure encountered but output generated till now : 706
		 232 2
	 Total Runtime Exception Counts  till now : 14
Successfully processed: 1704.01074v4.3

Processing file 11960 : 1808.04660v1.2
	 Total Error Counts till now : 720
	 Total Counts where failure encountered but output generated till now : 706
		 232 2
	 Total Runtime Exception Counts  till now : 14
Succ

Successfully processed: 1603.03181v1.3

Processing file 11990 : 1501.04706v3.3
	 Total Error Counts till now : 721
	 Total Counts where failure encountered but output generated till now : 707
		 232 2
	 Total Runtime Exception Counts  till now : 14
Successfully processed: 1501.04706v3.3

Processing file 11991 : 1605.02628v1.1
	 Total Error Counts till now : 721
	 Total Counts where failure encountered but output generated till now : 707
		 232 2
	 Total Runtime Exception Counts  till now : 14
Successfully processed: 1605.02628v1.1

Processing file 11992 : 1408.5530v1.1
	 Total Error Counts till now : 721
	 Total Counts where failure encountered but output generated till now : 707
		 232 2
	 Total Runtime Exception Counts  till now : 14
Successfully processed: 1408.5530v1.1

Processing file 11993 : 1711.00066v4.1
	 Total Error Counts till now : 721
	 Total Counts where failure encountered but output generated till now : 707
		 232 2
	 Total Runtime Exception Counts  till now : 14
Succes

In [18]:
# pprint(run_single_sample('1102.0674v1.1'))

In [19]:
import pickle
failures = pickle.load(open(dataset_type + "_failures.pickle", "rb"))
failures

{1: {'type': 'Not all unique rows and columns assigned at least one bbox',
  'files': ['1705.09728v1.2',
   '1401.0778v1.1',
   '1502.07994v1.5',
   '1406.2348v2.2',
   '1608.05852v1.5',
   '1710.09340v3.4',
   '1804.07961v1.1',
   '1709.10205v3.2',
   '0907.2993v1.1',
   '1704.04520v6.4',
   '1608.08940v1.1',
   '9912004v1.2',
   '1410.0471v1.2',
   '1502.07994v1.4',
   '1705.04971v1.2',
   '1510.08663v2.1',
   '1703.03091v1.9',
   '1104.3904v1.4',
   '1609.03136v1.1',
   '1510.08663v2.3',
   '1608.00293v1.7',
   '1705.07609v1.5',
   '0009011v1.30',
   '1708.01104v1.2',
   '1805.06960v2.7',
   '1709.10215v1.3',
   '1008.3314v1.4',
   '1704.08300v2.5',
   '1705.04971v1.1',
   '1209.0911v1.1',
   '1109.0545v1.5',
   '1502.07994v1.3',
   '1508.06010v1.3',
   '1102.5328v1.9',
   '1509.03547v2.6',
   '1710.09340v3.3',
   '1606.00548v1.1',
   '1011.2269v1.1',
   '1611.05250v2.4',
   '1607.02284v2.1',
   '1707.06068v1.3',
   '1805.02430v3.1',
   '1710.09340v3.1',
   '1805.06960v2.1',
   '150

In [20]:
def retry_failed_files(failures, failure_code):
    for file_prefix in failures[failure_code]['files']:
        try:
            process_sample(file_prefix)
            print("Processed " + file_prefix)
        except:
            print("Runtime exception in " + file_prefix)

In [21]:
# retry_failed_files(failures, 3)